In [104]:
import pandas as pd

url="https://archive.ics.uci.edu/ml/machine-learning-databases/00477/Real%20estate%20valuation%20data%20set.xlsx"
df=pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00477/Real%20estate%20valuation%20data%20set.xlsx",encoding='utf8')


In [105]:
column={i:list(df.columns)[i] for i in range(8)}

In [106]:
import numpy as np
df_test=df
columns_binary=['X1_1 early transaction','X1_2 late transaction','X2_1 younger age','X2_2 older age','X3_1 close to station','X3_2 far from station','X4_1 a few convinience store','X4_2 many convinience stores','X5_1 north','X5_2 south','X6_1 east','X6_2 west','Y_1 cheap','Y_2 expensive']
df_binary=pd.DataFrame(np.zeros(14*len(df)).reshape((len(df),14)),columns=columns_binary)

dict_col_bin={i:columns_binary[i] for i in range(14)}

In [ ]:

def mat2transactions(X, labels=[]):
    T = []
    for i in range(X.shape[0]):
        l = np.nonzero(X[i, :])[0].tolist()
        if labels:
            l = [labels[i] for i in l]
        T.append(l)
    return T

mat2transactions(df_test.to_numpy(),list(df_test.columns))

In [252]:
list(df.quantile(1/2))

[207.5, 2013.1666667, 16.1, 492.2313, 4.0, 24.9711, 121.53863, 38.45]

In [258]:
def print_apriori_rules(rules):
    frules = []
    for r in rules:
        for o in r.ordered_statistics:        
            conf = o.confidence
            supp = r.support
            lift=o.lift
            x = list( o.items_base ) 
            y = list( o.items_add ) 
            print("{%s} -> {%s}  (supp: %.3f, conf: %.3f)"%(x,y, supp, conf))
            frules.append( (x,y,supp,conf,len(x),lift) )
    return frules

def inspect_print_apriori_rules(rules):
    frules = []
    for r in rules:
        for o in r.ordered_statistics:        
            conf = o.confidence
            supp = r.support
            x = ", ".join( list( o.items_base ) )
            y = ", ".join( list( o.items_add ) )
            #print("{%s} -> {%s}  (supp: %.3f, conf: %.3f)"%(x,y, supp, conf))
            frules.append( (x,y) )
    return frules


In [259]:
import sys
import random
from apyori import apriori



def mat2transactions(X, labels=[]):
    T = []
    for i in range(X.shape[0]):
        l = np.nonzero(X[i, :])[0].tolist()
        if labels:
            l = [labels[i] for i in l]
        T.append(l)
    return T


counter=0
ls_rules=[]

df_binary=pd.DataFrame(np.zeros(14*len(df)).reshape((len(df),14)),columns=columns_binary)



while(counter<1):
    thresh_holds=[1/2 for i in range(8)]
    
    
    for j in range(1,8):
        thre=df_test.quantile(thresh_holds[j]).iloc[j]
        for i in range(len(df)):
            if(df_test.iloc[i].iat[j] < thre):
                df_binary[dict_col_bin[2*(j-1)]][i]=int(1)
                df_binary[dict_col_bin[2*(j-1)+1]][i]=int(0)

            else:
                df_binary[dict_col_bin[2*(j-1)]][i]=int(0)
                df_binary[dict_col_bin[2*(j-1)+1]][i]=int(1)

                
    dfb_test=df_binary
    #dfb_test.columns=['X1_1','X1_2','X2_1','X2_2','X3_1','X3_2','X4_1','X4_2','X5_1','X5_2','X6_1','X6_2','Y_1','Y_2']
    #dic_alph={i:dfb_test.columns[i] for i in range(14)}
    T= mat2transactions(dfb_test.to_numpy(),list(dfb_test.columns))

    
    rules = apriori(T, min_support=0.3,min_confidence=0.85,use_colnames=True)
    
    
    result=print_apriori_rules(rules)
    #result['length'] = result['itemsets'].apply(lambda x: len(x))
        
    counter+=1
    print("\r %d times / 10^100 (%.3f percent) %d Found"%(counter,counter/(10**5)*100,len(ls_rules)),end='')

{['X3_1 close to station']} -> {['X4_2 many convinience stores']}  (supp: 0.418, conf: 0.861)
{['X4_2 many convinience stores', 'X3_1 close to station']} -> {['X6_2 west']}  (supp: 0.367, conf: 0.879)
{['X6_2 west', 'X3_1 close to station']} -> {['X4_2 many convinience stores']}  (supp: 0.367, conf: 1.000)
{['X4_2 many convinience stores', 'X6_2 west']} -> {['X3_1 close to station']}  (supp: 0.367, conf: 0.894)
{['Y_2 expensive', 'X3_1 close to station']} -> {['X4_2 many convinience stores']}  (supp: 0.345, conf: 0.905)
{['Y_2 expensive', 'X6_2 west']} -> {['X3_1 close to station']}  (supp: 0.304, conf: 0.900)
{['X3_2 far from station', 'X4_1 a few convinience store']} -> {['Y_1 cheap']}  (supp: 0.331, conf: 0.884)
{['Y_1 cheap', 'X4_1 a few convinience store']} -> {['X3_2 far from station']}  (supp: 0.331, conf: 0.913)
{['X3_2 far from station', 'X5_1 north']} -> {['Y_1 cheap']}  (supp: 0.304, conf: 0.926)
{['Y_1 cheap', 'X5_1 north']} -> {['X3_2 far from station']}  (supp: 0.304, con

In [260]:
pdres=pd.DataFrame(result,columns=["Assumption","Conclusion","Support","Confidence","Length","Lift"]).sort_values("Support",ascending=False)
pdres

,Assumption,Conclusion,Support,Confidence,Length,Lift
0,[X3_1 close to station],[X4_2 many convinience stores],0.417874,0.860697,1,1.542547
1,"[X4_2 many convinience stores, X3_1 close to s...",[X6_2 west],0.367150,0.878613,2,1.740410
2,"[X6_2 west, X3_1 close to station]",[X4_2 many convinience stores],0.367150,1.000000,2,1.792208
3,"[X4_2 many convinience stores, X6_2 west]",[X3_1 close to station],0.367150,0.894118,2,1.841615
4,"[Y_2 expensive, X3_1 close to station]",[X4_2 many convinience stores],0.345411,0.905063,2,1.622061
10,"[Y_2 expensive, X6_2 west]",[X4_2 many convinience stores],0.333333,0.985714,2,1.766605
6,"[X3_2 far from station, X4_1 a few convinience...",[Y_1 cheap],0.330918,0.883871,2,1.767742
7,"[Y_1 cheap, X4_1 a few convinience store]",[X3_2 far from station],0.330918,0.913333,2,1.775211
5,"[Y_2 expensive, X6_2 west]",[X3_1 close to station],0.304348,0.900000,2,1.853731
8,"[X3_2 far from station, X5_1 north]",[Y_1 cheap],0.304348,0.926471,2,1.852941


In [229]:
ru=(apriori(dfb_test, min_support=(1/400)**5,min_confidence=(1/400)**5))
len(inspect_print_apriori_rules(ru))

173

In [141]:
list(rules)

[]

In [178]:
list(result)

[('', 'X1_1', 0.4830917874396135, 0.4830917874396135),
 ('', 'X1_2', 0.5169082125603864, 0.5169082125603864),
 ('', 'X2_1', 0.49516908212560384, 0.49516908212560384),
 ('', 'X2_2', 0.5048309178743962, 0.5048309178743962),
 ('', 'X3_1', 0.4855072463768116, 0.4855072463768116),
 ('', 'X3_2', 0.5144927536231884, 0.5144927536231884),
 ('', 'X4_1', 0.4420289855072464, 0.4420289855072464),
 ('', 'X4_2', 0.5579710144927537, 0.5579710144927537),
 ('', 'X5_1', 0.4927536231884058, 0.4927536231884058),
 ('', 'X5_2', 0.5072463768115942, 0.5072463768115942),
 ('', 'X6_1', 0.49516908212560384, 0.49516908212560384),
 ('', 'X6_2', 0.5048309178743962, 0.5048309178743962),
 ('', 'Y_1', 0.5, 0.5),
 ('', 'Y_2', 0.5, 0.5),
 ('X3_1', 'X4_2', 0.4178743961352657, 0.8606965174129354),
 ('X4_2', 'X3_1', 0.4178743961352657, 0.7489177489177489),
 ('X3_1', 'X5_2', 0.321256038647343, 0.6616915422885572),
 ('X5_2', 'X3_1', 0.321256038647343, 0.6333333333333333),
 ('X3_1', 'X6_2', 0.3671497584541063, 0.75621890547263

In [20]:
list(rules)

[]

In [92]:
df.quantile(1/3)

No                                         138.666667
X1 transaction date                       2013.000000
X2 house age                                12.766667
X3 distance to the nearest MRT station     372.462333
X4 number of convenience stores              3.000000
X5 latitude                                 24.965150
X6 longitude                               121.534080
Y house price of unit area                  31.300000
Name: 0.3333333333333333, dtype: float64

In [439]:
len(df)

414

In [86]:
df_binary

,X1_1 early transaction,X1_2 late transaction,X2_1 younger age,X2_2 older age,X3_1 close to station,X3_2 far from station,X4_1 a few convinience store,X4_2 many convinience stores,X5_1 north,X5_2 south,X6_1 east,X6_2 west,Y_1 cheap,Y_2 expensive
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
410,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
411,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
